In [1]:
import pandas as pd

In [2]:
#read the csv review dataset
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='PcBfyIwGat2itUjt7RMWnbFW9Wt2mY8r2RCDr8320vDI',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'understandingguestperceptionsofho-donotdelete-pr-h52h40bibzzljq'
object_key = 'hotel_reviews.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

trip = pd.read_csv(body)
trip.head()


,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [3]:
# Let's create a new data frame
 
trip = trip[(trip['Rating']==5)|(trip['Rating']==2)|(trip['Rating']==1)][['Review','Rating']]

# Lets modify the Rating column
trip['Rating'] = trip['Rating'].apply(lambda rating: 'Pos' if rating==5 else 'Neg')

In [4]:
# reseting the index because after removing some rows, the index gets crowded
trip.reset_index(inplace=True)
trip.head()

,index,Review,Rating
0,1,ok nothing special charge diamond member hilto...,Neg
1,3,"unique, great stay, wonderful time hotel monac...",Pos
2,4,"great stay great stay, went seahawk game aweso...",Pos
3,5,love monaco staff husband stayed hotel crazy w...,Pos
4,6,"cozy stay rainy city, husband spent 7 nights m...",Pos


In [5]:
trip['Rating'].value_counts()

Pos    9054
Neg    3214
Name: Rating, dtype: int64

In [6]:
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 71.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
#Data cleaning and preprocessing

import re
import nltk

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/wsuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/wsuser/nltk_data...


True

In [9]:
# Lemmatization object
ps = WordNetLemmatizer()
corpus = []

In [10]:
# Text preprocessing
# keep only text based
# lower all the letters
# split the words
for i in range(0,len(trip)):
    review = re.sub('[^a-zA-Z]'," ",trip['Review'][i])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [11]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='PcBfyIwGat2itUjt7RMWnbFW9Wt2mY8r2RCDr8320vDI',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'understandingguestperceptionsofho-donotdelete-pr-h52h40bibzzljq'
object_key = 'tdata.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

trip.to_csv('df_data_1.csv', index = False)


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [13]:
X = cv.fit_transform(corpus).toarray()

In [14]:
X.shape

(12268, 34569)

In [15]:
y = pd.get_dummies(trip['Rating'])
y = y.iloc[:,1].values
y

array([0, 1, 1, ..., 0, 0, 0], dtype=uint8)

In [16]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=3)

Naive Bayes

In [17]:
#Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB().fit(X_train,y_train)

# for the accuracy
model1.score(X_test,y_test)

y_pred = model1.predict(X_test)

In [18]:
#compare y test and y_pred
#confusion matrix is a 2x2 matrix and it tells,
#how many number of elements are correctly predicted.

from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test,y_pred)

confusion_m

array([[ 574,   69],
       [  44, 1767]])

In [19]:
#checking accuracy score

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)

accuracy

0.9539527302363489

In [20]:
#checking precision score

from sklearn.metrics import precision_score
precision_score(y_test,y_pred)

0.9624183006535948

In [21]:
#checking recall score

from sklearn.metrics import recall_score
recall_score(y_test,y_pred)

0.9757040309221424

In [22]:
#checking f-beta score

from sklearn.metrics import fbeta_score
fbeta_score(y_test,y_pred,beta=1)

0.9690156292843433

In [23]:
# Checking the training model with custom input data
message = "the hotel was nice and comfy"
data = [message]
vect = cv.transform(data).toarray()
my_prediction = model1.predict(vect)
my_prediction_prob = model1.predict_proba(vect)

if my_prediction==1:
    print("Positive")
    print(my_prediction_prob[0][1])
else:
    print("Negative")
    print(my_prediction_prob[0][0])

Positive
0.9689465878406804


In [24]:
message = "the hotel was bad and the staff was rude"
data = [message]
vect = cv.transform(data).toarray()
my_prediction = model1.predict(vect)
my_prediction_prob = model1.predict_proba(vect)

if my_prediction==1:
    print("Positive")
    print(my_prediction_prob[0][1])
else:
    print("Negative")
    print(my_prediction_prob[0][0])

Negative
0.8445454685022524


Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='liblinear')

In [26]:
model2 = logreg.fit(X_train, y_train)

In [27]:
y_pred_class = logreg.predict(X_test)

In [28]:
from sklearn import metrics

In [29]:
metrics.accuracy_score(y_test, y_pred)

0.9539527302363489

In [30]:
metrics.precision_score(y_test, y_pred)

0.9624183006535948

In [31]:
metrics.precision_score(y_test, y_pred)

0.9624183006535948

In [32]:
metrics.recall_score(y_test, y_pred)

0.9757040309221424

In [33]:
metrics.fbeta_score(y_test, y_pred, beta=1)

0.9690156292843433

In [34]:
metrics.roc_auc_score(y_test, y_pred)

0.9341972720707136

In [35]:
pip install metrics

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.7/841.7 kB 60.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for metrics: filename=metrics-0.3.3-py2.py3-none-any.whl size=17808 sha256=bb08669fedd3de55c472d5f150d7a4e4945d535e5ee9fc0791b47c81fe1c3786
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/29/7a/e7/1175d9ff10607b8f02aa37c32392cb28cdda0aea8fcc2b514b
  Created wheel for pathspec: filename=pathspec-0.5.5-py3-none-any.whl size=24233 sha256=fa391a8744656a77a41064dbe34301ddc45538a5072013755123c48355f521e4
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/28/d4/11/01ccd690e97b06874998aa554a8b2610c361900d2a7e038300
Successfully built metrics pathspec
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2
  Attempting uninstall: pathspec
    Found existing installation: pathspec 0.9.0
    Uninstalling path

In [36]:
metrics.confusion_matrix(y_test, y_pred)

array([[ 574,   69],
       [  44, 1767]])

In [37]:
message = "cozy stay rainy city"
data = [message]
vect = cv.transform(data).toarray()
my_prediction = model2.predict(vect)
my_prediction_prob = model2.predict_proba(vect)

if my_prediction==1:
    print("Positive")
    print(my_prediction_prob[0][1])
else:
    print("Negative")
    print(my_prediction_prob[0][0])

Positive
0.6713142877649974


In [38]:
message = "the hotel was bad and the staff was rude"
data = [message]
vect = cv.transform(data).toarray()
my_prediction = model2.predict(vect)
my_prediction_prob = model2.predict_proba(vect)

if my_prediction==1:
    print("Positive")
    print(my_prediction_prob[0][1])
else:
    print("Negative")
    print(my_prediction_prob[0][0])

Negative
0.8996652277103434


In [39]:
# Dump the machine learning model outsite so you can use outsite and not retrain again and again
import pickle

#pickle file for logistic regression

filename = 'logistic_regression_model.pkl'
pickle.dump(model2, open(filename, 'wb'))
pickle.dump(cv, open('tranform_logistic.pkl', 'wb'))

In [40]:
#pickle file for naive bayes

filename = 'naive_bayes_model.pkl'
pickle.dump(model1, open(filename, 'wb'))
pickle.dump(cv, open('transform_naive.pkl', 'wb'))

In [41]:
!pip install -U ibm-watson-machine-learning
!tar -zcvf hotel_reviews_model.tgz parkinson.pkl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.280
    Uninstalling ibm-watson-machine-learning-1.0.280:
      Successfully uninstalled ibm-watson-machine-learning-1.0.280
tar: parkinson.pkl: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [45]:
from ibm_watson_machine_learning import APIClient
wml_credentials={"url":"https://us-south.ml.cloud.ibm.com", 
                 "apikey": "8BhFIxrwf72Px-O8ISOSgsCj43vy4EUmweTYxJmr9fan"
                }
client=APIClient(wml_credentials)

In [46]:
def guide_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources']if item['entity']["name"]==space_name)['metadata']['id'])

In [48]:
space_uid=guide_from_space_name(client,'hotel_deploy')
print("Space UID =" + space_uid)

Space UID =a57419a5-04d7-4958-85ad-7593d1c31baa


In [49]:
client.set.default_space(space_uid)


'SUCCESS'

In [50]:
client.software_specifications.list()


-----------------------------  ------------------------------------  ----  ------------------  -------------------------------
NAME                           ID                                    TYPE  STATE               REPLACEMENT
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  unsupported         runtime-22.1-py3.9
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base  not_provided
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base  not_provided
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  unsupported         runtime-22.1-py3.9
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base  unsupported
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base  supported
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base  unsupported         runtime-22.1-py3.9
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base  n

In [54]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [56]:
model_details=client.repository.store_model(model='hotel_reviews_model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"hotel_model",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_id(model_details)

In [57]:
model_id

'abd5826b-6a7b-4e14-8edf-d0e276c63dd8'

In [63]:
client.repository.download("abd5826b-6a7b-4e14-8edf-d0e276c63dd8","hotel.tar.gz")


File with name: 'hotel.tar.gz' already exists.


WMLClientError: File with name: 'hotel.tar.gz' already exists.

In [62]:
client.repository.download("abd5826b-6a7b-4e14-8edf-d0e276c63dd8","hotel_reviews_lr_model.tar.gz")


File with name: 'hotel_reviews_lr_model.tar.gz' already exists.


WMLClientError: File with name: 'hotel_reviews_lr_model.tar.gz' already exists.

In [65]:
client.repository.download("abd5826b-6a7b-4e14-8edf-d0e276c63dd8","hotel_reviews_tl_model.tar.gz")


Successfully saved model content to file: 'hotel_reviews_tl_model.tar.gz'


'/home/wsuser/work/hotel_reviews_tl_model.tar.gz'

In [66]:
client.repository.download("abd5826b-6a7b-4e14-8edf-d0e276c63dd8","hotel_reviews_nb_model1.tar.gz")


Successfully saved model content to file: 'hotel_reviews_nb_model1.tar.gz'


'/home/wsuser/work/hotel_reviews_nb_model1.tar.gz'

In [67]:
client.repository.download("abd5826b-6a7b-4e14-8edf-d0e276c63dd8","hotel_reviews_tn_model.tar.gz")


Successfully saved model content to file: 'hotel_reviews_tn_model.tar.gz'


'/home/wsuser/work/hotel_reviews_tn_model.tar.gz'